In [12]:
print('ok')

ok


In [13]:
%pwd

'c:\\Users\\syedm\\Desktop\\Generative AI Medical Chat bot\\End-to-End-Medical-Chatbot-Hosted-on-Cloud-Platform-Vector-Base-\\research'

In [14]:
#  changing directory to main directory
import os 
os.chdir('../')

In [15]:
%pwd

'c:\\Users\\syedm\\Desktop\\Generative AI Medical Chat bot\\End-to-End-Medical-Chatbot-Hosted-on-Cloud-Platform-Vector-Base-'

In [16]:
#These are document loaders in LangChain, used to import data into your chatbot from files like PDFs.

# PyPDFLoader : Purpose: Loads content from a single PDF file.
# DirectoryLoader Purpose: Loads multiple documents from a folder.
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# text splitter that breaks large documents into smaller chunks
# RecursiveCharacterTextSplitter: Splits long text into manageable chunks (e.g., 500–1000 characters)


In [17]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,  glob="*.pdf", loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents # it returns  the list of all extracted documents (text and metadata).


# DirectoryLoader: Loads all .pdf files in the folder path data.
# glob="*.pdf": Filters only PDF files.
# loader_cls=PyPDFLoader: Uses the PyPDFLoader to read each PDF file.

In [18]:
#  calls the fucn and passes data path
extarcted_data = load_pdf_file(data='Data/')


In [19]:
# PERFORMING CHUNKING OPERATIONS :
def text_splitter(extarcted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400 , chunk_overlap = 10)
    text_chunks = text_splitter.split_documents(extarcted_data)
    return  text_chunks

 

In [20]:
text_chunks = text_splitter(extarcted_data)
print('Length of Text Chunks', len(text_chunks))

Length of Text Chunks 8681


In [21]:
# EMBEDING MODEL FOR VECTOR EMBEDING FROM HUGGING FACE  and using all-MiniLM-L6-v2

from langchain.embeddings import HuggingFaceEmbeddings

#This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional 
# dense vector space and can be used for tasks like clustering or semantic search.



In [22]:
# Download the Embeding model from Hugging face

def download_hugging_face_embedings():
    embedings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedings

In [23]:
embedings = download_hugging_face_embedings()


C:\Users\syedm\AppData\Local\Temp\ipykernel_9128\3695427722.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [24]:
# checking whether our text is converting into vector embedings or not
query_result = embedings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [25]:
# importing environement so that can access pine cone api key 
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
import os
#  getting key fron env  variable and save it in a variale and call that in fucntion
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPEN_AI_API_KEY = os.environ.get('OPEN_AI_API_KEY')
OPEN_AI_BASE_URL = os.environ.get('OPEN_AI_BASE_URL')


In [27]:
# Storing these vectors inside pinecone , we ,manully create indexes in piencone on web and in python also ,  we prefer python

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 


{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-oftdilp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [28]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPEN_AI_API_KEY"] = OPEN_AI_API_KEY
os.environ["OPEN_AI_BASE_URL"] = OPEN_AI_BASE_URL


In [29]:
#  Now convert the chunks into vector embeding and store it in piencone database .
# This creates a new Pinecone vector store and immediately:
# Embeds your text chunks
# Upserts (stores) them into the Pinecone index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents= text_chunks,
    index_name= index_name,
    embedding= embedings, 
)


In [31]:
# LOAD EXISTING INDEX  (This loads a Pinecone index that already has vector embeddings stored inside it.) 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedings
)

# THIS IS DONE DUE TO :
# Now you just want to query the index, not upload again
# You want to perform a search (e.g., similarity search) on the existing data

In [32]:
# TESTING 
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

# INPUT 
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='8dd59802-3474-4a35-8c6f-4683e655120e', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 737.0, 'page_label': '738', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 21352\nFood poisoning\nAcne folliculitis. (Custom Medical Stock Photo. Reproduced by\npermission.)'),
 Document(id='ff8b440f-1081-4d42-a67d-cf0678b5bebe', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 297.0, 'page_label': '298', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='tion, can make the skin more susceptible to ICD.\nAllergic contact dermatitis (A

In [33]:
#  NOW WE WILL CONNECT THE LLM WITH THIS KNOWLEDGE BASE  , AND PEROFRM SEMANTIC SEARCHA NS ANSWER USER 
# QUERRY  

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(
    model="deepseek/deepseek-r1:free",
    openai_api_base= OPEN_AI_BASE_URL,
    openai_api_key= OPEN_AI_API_KEY ,
    temperature=0.4,
    max_tokens=500
)

In [34]:
# Test it
response = llm([HumanMessage(content="Tell me about imran khan")])
print(response.content)
response


Imran Khan, born on October 5, 1952, in Lahore, Pakistan, is a multifaceted figure known for his achievements in cricket, philanthropy, and politics. Here's an overview of his life and career:

### **Early Life and Education**
Imran Khan grew up


AIMessage(content="Imran Khan, born on October 5, 1952, in Lahore, Pakistan, is a multifaceted figure known for his achievements in cricket, philanthropy, and politics. Here's an overview of his life and career:\n\n### **Early Life and Education**\nImran Khan grew up", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 500, 'prompt_tokens': 13, 'total_tokens': 513, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None}, id='run--9f3bd2b8-1ffa-40af-863b-4d06aa3f2d1e-0')

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [38]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism both?"})
print(response["answer"])

Acromegaly and gigantism are both caused by excessive growth hormone (GH) production, typically from a pituitary gland tumor. Gigantism occurs when excess GH arises in childhood before growth plate closure, leading to abnormal height. Acromegaly occurs in adulthood after growth plates close, causing enlarged bones and tissues, especially in hands, feet, and face.
